Importing MobileNetV2

In [2]:
from keras.applications.mobilenet_v2 import MobileNetV2

2023-03-04 17:36:22.548033: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 17:36:23.218825: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-04 17:36:23.218860: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-04 17:36:25.058248: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
model = MobileNetV2(weights='imagenet')

2023-03-04 17:36:27.065786: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-04 17:36:27.065976: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-04 17:36:27.066032: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-04 17:36:27.066078: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-04 17:36:27.066123: W tensorf

### Data Preprocessing:

Loading the data and generating labels

In [4]:
import pandas as pd
import numpy as np
from imageio import imread
from skimage.transform import resize
from keras.applications.mobilenet_v2 import preprocess_input

In [5]:
# Read the csv file into the DataFrame
df = pd.read_csv('Labelled_Data.csv')

# Get the number of rows in the DataFrame
num_of_images = len(df)

# Print the number of images
print(f"Number of Labelled Images: {num_of_images}")

Number of Labelled Images: 730


In [6]:
data = np.empty((num_of_images, 224, 224, 3)) #num_of_images is the number of images that I want to train

for i in range(num_of_images):
  image_name = df.iloc[i, 1]
  im = imread(f'Training-Images/{image_name}')
  im = preprocess_input(im)
  im = resize(im, output_shape=(224, 224))
  data[i] = im



/tmp/ipykernel_1879/3710380246.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  im = imread(f'Training-Images/{image_name}')


Tidy up the data and create the matrix

|Categories | Number Representation |
| --- | --- |
| Cosmetics and Medicine | 0 |
| Electronics | 1 |
| Textile, Clothing & Shoes | 2 |
| Furniture & Home appliances | 3 |
| Toys  | 4 |
| Books & Movies | 5 |


In [7]:
"""
Generating Labels
"""
category_map = {
    "Cosmetics & Medicine": 0,
    "Electronics": 1,
    "Textile, Clothing & Shoes": 2,
    "Furniture & Home appliances": 3,
    "Toys ": 4,
    "Books & Movies": 5
}


labels = np.empty(num_of_images, dtype=int)

for index, row in df.iterrows():
    category = row[2]
    labels[index] = category_map[category]


Generating predictions for items, without transfer learning

In [8]:
from keras.applications.mobilenet_v2 import decode_predictions

In [9]:
predictions = model.predict(data) 

for decoded_prediction in decode_predictions(predictions, top=1):
    for name, desc, score in decoded_prediction:
        print('- {} ({:.2f}%)'.format(desc, 100*score))

23/23 [==============================] - 13s 518ms/step
- jean (67.30%)
- jersey (69.99%)
- mailbag (28.82%)
- mailbag (55.82%)
- window_shade (60.51%)
- tray (34.82%)
- space_heater (27.92%)
- modem (29.58%)
- coffee_mug (73.68%)
- folding_chair (16.72%)
- pajama (20.89%)
- tray (85.62%)
- bulletproof_vest (69.97%)
- swimming_trunks (36.38%)
- mailbag (30.02%)
- vacuum (40.32%)
- scale (80.76%)
- wooden_spoon (89.50%)
- scabbard (23.17%)
- Band_Aid (44.12%)
- toilet_seat (32.25%)
- sunscreen (23.22%)
- carton (24.10%)
- cowboy_boot (19.62%)
- Dutch_oven (19.14%)
- carton (29.47%)
- sweatshirt (53.46%)
- cowboy_boot (43.38%)
- apron (29.28%)
- sweatshirt (55.30%)
- water_bottle (6.50%)
- sweatshirt (29.95%)
- bulletproof_vest (24.97%)
- apron (14.04%)
- jersey (32.61%)
- umbrella (16.26%)
- jersey (30.53%)
- sweatshirt (52.23%)
- umbrella (54.12%)
- cowboy_boot (89.93%)
- running_shoe (85.05%)
- running_shoe (53.11%)
- running_shoe (37.28%)
- clog (73.20%)
- sweatshirt (97.71%)
- ponch

Modifying the Model

In [10]:
from keras.layers import Dense
from keras import Model

In [11]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [12]:
label_output = Dense(6, activation='softmax')
label_output = label_output(model.layers[-2].output) #this is a keras tensor, to connect the nodes to our putput nodes
label_input = model.input
label_model = Model(inputs=label_input, outputs=label_output)

for layer in label_model.layers[:-1]:
    layer.trainable = False

Compiling the Model

In [13]:
label_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

Training the Model

In [14]:
label_model.fit(
    x=data,
    y=labels,
    epochs=20,
    verbose=2
)

Epoch 1/20
23/23 - 13s - loss: 0.8112 - accuracy: 0.7699 - 13s/epoch - 572ms/step
Epoch 2/20
23/23 - 12s - loss: 0.3146 - accuracy: 0.9260 - 12s/epoch - 506ms/step
Epoch 3/20
23/23 - 11s - loss: 0.1969 - accuracy: 0.9466 - 11s/epoch - 476ms/step
Epoch 4/20
23/23 - 11s - loss: 0.1415 - accuracy: 0.9616 - 11s/epoch - 466ms/step
Epoch 5/20
23/23 - 11s - loss: 0.1026 - accuracy: 0.9753 - 11s/epoch - 475ms/step
Epoch 6/20
23/23 - 11s - loss: 0.0828 - accuracy: 0.9822 - 11s/epoch - 483ms/step
Epoch 7/20
23/23 - 11s - loss: 0.0647 - accuracy: 0.9836 - 11s/epoch - 490ms/step
Epoch 8/20
23/23 - 11s - loss: 0.0528 - accuracy: 0.9890 - 11s/epoch - 488ms/step
Epoch 9/20
23/23 - 11s - loss: 0.0428 - accuracy: 0.9932 - 11s/epoch - 469ms/step
Epoch 10/20
23/23 - 11s - loss: 0.0356 - accuracy: 0.9959 - 11s/epoch - 481ms/step
Epoch 11/20
23/23 - 11s - loss: 0.0302 - accuracy: 1.0000 - 11s/epoch - 497ms/step
Epoch 12/20
23/23 - 11s - loss: 0.0265 - accuracy: 1.0000 - 11s/epoch - 464ms/step
Epoch 13/20
2

Generating predictions for training data

In [15]:
predictions = label_model.predict(data)

23/23 [==============================] - 12s 509ms/step


In [16]:
predictions.shape

(730, 6)

In [17]:
np.argmax(predictions, axis=1)

array([2, 2, 2, 2, 3, 5, 3, 3, 3, 2, 2, 3, 2, 2, 2, 1, 3, 3, 3, 0, 3, 0,
       5, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 5, 0, 2,
       2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2, 2,

Generating separate training and validation sets

In [18]:
# separate training and validation sets randomly. 80% training, 20% validation
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    data, labels, test_size=0.2, random_state=42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape


((584, 224, 224, 3), (146, 224, 224, 3), (584,), (146,))

In [19]:
label_model.fit(
    x=X_train,
    y=y_train,
    epochs=20,
    verbose=2,
    validation_data=(X_val, y_val)
)

predictions = label_model.predict(X_val)

predictions.shape




Epoch 1/20
19/19 - 12s - loss: 0.0106 - accuracy: 1.0000 - val_loss: 0.0103 - val_accuracy: 1.0000 - 12s/epoch - 625ms/step
Epoch 2/20
19/19 - 11s - loss: 0.0098 - accuracy: 1.0000 - val_loss: 0.0110 - val_accuracy: 1.0000 - 11s/epoch - 554ms/step
Epoch 3/20
19/19 - 11s - loss: 0.0090 - accuracy: 1.0000 - val_loss: 0.0105 - val_accuracy: 1.0000 - 11s/epoch - 558ms/step
Epoch 4/20
19/19 - 10s - loss: 0.0084 - accuracy: 1.0000 - val_loss: 0.0112 - val_accuracy: 1.0000 - 10s/epoch - 542ms/step
Epoch 5/20
19/19 - 10s - loss: 0.0079 - accuracy: 1.0000 - val_loss: 0.0113 - val_accuracy: 1.0000 - 10s/epoch - 526ms/step
Epoch 6/20
19/19 - 10s - loss: 0.0073 - accuracy: 1.0000 - val_loss: 0.0115 - val_accuracy: 1.0000 - 10s/epoch - 518ms/step
Epoch 7/20
19/19 - 10s - loss: 0.0069 - accuracy: 1.0000 - val_loss: 0.0117 - val_accuracy: 1.0000 - 10s/epoch - 518ms/step
Epoch 8/20
19/19 - 11s - loss: 0.0066 - accuracy: 1.0000 - val_loss: 0.0116 - val_accuracy: 1.0000 - 11s/epoch - 576ms/step
Epoch 9/

(146, 6)

In [20]:
np.argmax(predictions, axis=1)


array([2, 2, 2, 5, 2, 2, 2, 1, 2, 5, 5, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 5, 5, 2, 2, 2, 4, 2, 5, 2, 2, 2, 5, 2, 2, 2,
       2, 2, 2, 5, 2, 2, 2, 2, 5, 5, 2, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 5,
       5, 2, 2, 0, 2, 5, 2, 2, 2, 2, 2, 4, 5, 4, 2, 5, 5, 5, 2, 2, 2, 2,
       2, 2, 5, 4, 2, 2, 2, 5, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2, 2, 5, 2, 2,
       2, 2, 2, 2, 2, 5, 2, 4, 2, 2, 2, 2, 2, 0, 2, 5, 5, 2, 5, 5, 2, 2,
       2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [21]:
# now, check the percentage of accuracy of the model
from sklearn.metrics import accuracy_score

accuracy_score(y_val, np.argmax(predictions, axis=1))


1.0

In [24]:
from sklearn.metrics import f1_score

print("Image Classification using MobileNetV2")

# Print the number of images
print(f"Number of Labelled Images: {num_of_images}")

# Print Summary of the model
label_model.summary()

# Print the accuracy of the model
print(f"Accuracy of the model: {accuracy_score(y_val, np.argmax(predictions, axis=1))}")

# Overview of the results:
print("Overview of the results:")
for i in range(len(predictions)):
    print(f"Image {i+1}: Actual label: {y_val[i]}, Predicted label: {np.argmax(predictions[i])}")


# Print the f-measure of the model
print(f"F-measure of the model: {f1_score(y_val, np.argmax(predictions, axis=1), average='macro')}")

# Print the confusion matrix of the model
from sklearn.metrics import confusion_matrix
print(f"Confusion Matrix of the model: {confusion_matrix(y_val, np.argmax(predictions, axis=1))}")

# Print the classification report of the model
from sklearn.metrics import classification_report
print(f"Classification Report of the model: {classification_report(y_val, np.argmax(predictions, axis=1))}")

# Print the precision of the model
from sklearn.metrics import precision_score
print(f"Precision of the model: {precision_score(y_val, np.argmax(predictions, axis=1), average='macro')}")

# Print the recall of the model
from sklearn.metrics import recall_score
print(f"Recall of the model: {recall_score(y_val, np.argmax(predictions, axis=1), average='macro')}")


Image Classification using MobileNetV2
Number of Labelled Images: 730
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                        

In [35]:
# Summary of the results:
print("Classification of Images using MobileNetV2")
print("Summary of the results:")
print(f"Number of Labelled Images: {num_of_images}")
print(f"Number of Training Images: {len(X_train)}")
print(f"Number of Validation Images: {len(X_val)}")
print(f"Number of Epochs: 20")
print(f"Accuracy of the model: {accuracy_score(y_val, np.argmax(predictions, axis=1))}")
print(f"F-measure of the model: {f1_score(y_val, np.argmax(predictions, axis=1), average='macro')}")
print(f"Confusion Matrix of the model:\n\n {confusion_matrix(y_val, np.argmax(predictions, axis=1))}")
print(f"Classification Report of the model:\n\n {classification_report(y_val, np.argmax(predictions, axis=1))}")
print(f"Precision of the model: {precision_score(y_val, np.argmax(predictions, axis=1), average='macro')}")
print(f"Recall of the model: {recall_score(y_val, np.argmax(predictions, axis=1), average='macro')}")


Classification of Images using MobileNetV2
Summary of the results:
Number of Labelled Images: 730
Number of Training Images: 584
Number of Validation Images: 146
Number of Epochs: 20
Accuracy of the model: 1.0
F-measure of the model: 1.0
Confusion Matrix of the model:

 [[  2   0   0   0   0]
 [  0   1   0   0   0]
 [  0   0 109   0   0]
 [  0   0   0   5   0]
 [  0   0   0   0  29]]
Classification Report of the model:

               precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00       109
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00        29

    accuracy                           1.00       146
   macro avg       1.00      1.00      1.00       146
weighted avg       1.00      1.00      1.00       146

Precision of the model: 1.0
Recall of the model: 1.0
